In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from skimage.feature import hog

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
train_features = extract_HOG_features(x_train)
test_features = extract_HOG_features(x_test)

(2025,)

In [ ]:
train_features.shape

(60000, 2025)

In [ ]:
model_hog = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(2025)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.25),
  # tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(10, activation = 'softmax')
])

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.25),
  # tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(10, activation = 'softmax')
])

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# Compile model with the loss function of categorical cross entropy 
# Categorical cross entropy is most commonly used for classification problems
model.compile(loss=loss_fn, optimizer="adam", metrics=["accuracy"])
## Print the summary of the model
# model.summary()

In [ ]:
# This callback monitors a quantity and if no improvement is seen for a 'patience' number of epochs, the learning rate is reduced.
reduceLearningRate = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, min_lr=0.0001)
# Stop training when a monitored metric has stopped improving.
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

# Train model with .fit function
history = model.fit(
    # Pass train_data and train_labels (the pre-process data saved before)
    x_train,
    y_train,
    validation_split=0.2,
    # validation_data=(x_test, y_test),
    epochs=30,
    batch_size=100,
    # Pass callbacks to Early stopping by two checkpoints
    # Read Early stopping method for more detail
    callbacks=[reduceLearningRate, earlyStopping]
)
model.save(r'/content/drive/MyDrive/Machine Learning/ANN_MNIST/Model/test.h5')

In [ ]:
## Plot loss and val_loss frequency in each epoch 
plt.plot(history.history["loss"], 'b', label="Train Loss")
plt.plot(history.history["val_loss"], 'r', label="Validation loss")
plt.axis([0, 19, 0, 0.5])
plt.xlabel("Epoch")
plt.ylabel("Frequency")
plt.legend()
plt.show()
plt.style.use('default')

In [ ]:
predictions = model.predict(x_test)
actual = y_test
## Pass testLabels and predictions to confusion matrix
matrix = metrics.confusion_matrix(
    actual,
    np.argmax(predictions, axis = 1)
)
## Plot confusion matrix with matplotlib
labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

DataFrameMatrix = pd.DataFrame(matrix, columns=np.unique(labels), index = np.unique(labels))
DataFrameMatrix.index.name = 'Actual'
DataFrameMatrix.columns.name = 'Predict'
plt.figure(figsize = (10,10))
sn.set(font_scale=1.5) 
sn.heatmap(DataFrameMatrix, cmap="Reds", annot=True, annot_kws={"size": 10}, fmt="d")

In [ ]:
print(metrics.classification_report(actual, np.argmax(predictions, axis = 1), labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))
